# Download weight

In [2]:
# Delete all files and folders in the current directory. Use with caution!
import shutil
shutil.rmtree('./', ignore_errors=True)

In [3]:
!git clone https://github.com/BinhPQ2/Han_Nom_OCR.git .
!git clone https://github.com/THU-MIG/yolov10.git

Cloning into '.'...
remote: Enumerating objects: 148, done.
remote: Counting objects: 100% (148/148), done.
remote: Compressing objects: 100% (126/126), done.
remote: Total 148 (delta 15), reused 144 (delta 14), pack-reused 0 (from 0)
Receiving objects: 100% (148/148), 8.85 MiB | 22.00 MiB/s, done.
Resolving deltas: 100% (15/15), done.
Cloning into 'yolov10'...
remote: Enumerating objects: 20329, done.
remote: Counting objects: 100% (2443/2443), done.
remote: Compressing objects: 100% (245/245), done.
remote: Total 20329 (delta 2315), reused 2198 (delta 2198), pack-reused 17886 (from 1)
Receiving objects: 100% (20329/20329), 11.14 MiB | 18.38 MiB/s, done.
Resolving deltas: 100% (14335/14335), done.


In [5]:
!gdown --id 11VxsyUVmdH8yD3MtQT-t4I2q9YIvkp3w -O file.zip
!unzip -q file.zip

/usr/local/lib/python3.10/dist-packages/gdown/__main__.py:132: FutureWarning: Option `--id` was deprecated in version 4.3.1 and will be removed in 5.0. You don't need to pass it anymore to use a file ID.
  warnings.warn(
Downloading...
From (original): https://drive.google.com/uc?id=11VxsyUVmdH8yD3MtQT-t4I2q9YIvkp3w
From (redirected): https://drive.google.com/uc?id=11VxsyUVmdH8yD3MtQT-t4I2q9YIvkp3w&confirm=t&uuid=567f0595-ea17-4335-8284-dd852b9a1e41
To: /content/file.zip
100% 262M/262M [00:07<00:00, 35.3MB/s]


# Detection

In [8]:
!pip install -q -r requirements.txt
!pip install -q -e ./yolov10/
# !pip install supervision -q # already included in the requirement.txt
# !pip install ultralytics -q

  Installing build dependencies ... done
  Checking if build backend supports build_editable ... done
  Getting requirements to build editable ... done
  Preparing editable metadata (pyproject.toml) ... done
  Building editable for ultralytics (pyproject.toml) ... done


In [2]:
!pip show ultralytics

Name: ultralytics
Version: 8.1.34
Summary: Ultralytics YOLOv8 for SOTA object detection, multi-object tracking, instance segmentation, pose estimation and image classification.
Home-page: 
Author: Glenn Jocher, Ayush Chaurasia, Jing Qiu
Author-email: 
License: AGPL-3.0
Location: /usr/local/lib/python3.10/dist-packages
Editable project location: /content/yolov10
Requires: matplotlib, opencv-python, pandas, pillow, psutil, py-cpuinfo, pyyaml, requests, scipy, seaborn, thop, torch, torchvision, tqdm
Required-by: 


In [1]:
import os
import random
import numpy as np
from tqdm import tqdm
import pickle
import cv2
import supervision as sv
import matplotlib.pyplot as plt
from ultralytics import YOLOv10 # rerun session if it has error in this library

In [3]:
def save_crop(result, save_path):
    os.makedirs(save_path, exist_ok=True)

    conf_s = result.boxes.conf.detach().cpu().numpy()
    xyxy_s = result.boxes.xyxy.detach().cpu().numpy()
    idx_s = [i for i in range(len(conf_s))]
    img_name = os.path.basename(result.path)
    img = result.orig_img
    for idx, conf, xyxy in zip(idx_s, conf_s, xyxy_s):
        # name = f"oder_{idx}__conf_{conf}__name_{img_name}"
        name = img_name

        height=result.orig_img.shape[0]
        width=result.orig_img.shape[1]

        height_extend_ratio = 0.05
        width_extend_ratio = 0.1

        x0, y0, x1, y1 = xyxy.tolist()
        x0 = int(max(0, x0 - width_extend_ratio*abs(x1 - x0)))
        x1 = int(min(width, x1 + width_extend_ratio*abs(x1 - x0)))
        y0 = int(max(0, y0 - height_extend_ratio*abs(y1 - y0)))
        y1 = int(min(height, y1 + height_extend_ratio*abs(y1 - y0)))
        crop_image = img[y0:y1, x0:x1]

        cv2.imwrite(os.path.join(save_path, name), crop_image)

In [6]:
input_image_folder = "./input_image"
save_path = "./output_detection"
model = YOLOv10('./weight/detection_yolov10.pt')

results = model.predict(source=input_image_folder, conf=0.5, stream=True, save_txt=True)
for idx, result in enumerate(results):
    save_crop(result, save_path)


image 1/23 /content/input_image/02b02efc5d6b4579a0cae31e1e362555.jpg: 640x480 1 couplet, 1617.7ms
image 2/23 /content/input_image/0ff04276ca994b12a50f51f88a9373fc.jpg: 640x352 1 couplet, 779.5ms
image 3/23 /content/input_image/1a2291615b4e4c288a152b4886ac300e.jpg: 640x480 1 couplet, 997.1ms
image 4/23 /content/input_image/1a5634d0281b4b1aac6e616826c30d4b.jpg: 640x320 2 couplets, 681.4ms
image 5/23 /content/input_image/1a7296dbb22748e49b656df5cc6743b5.jpg: 640x288 2 couplets, 619.2ms
image 6/23 /content/input_image/1b094caec3984b8eb1c929b60a8d2b08.jpg: 640x480 4 couplets, 967.5ms
image 7/23 /content/input_image/1b648fd456b54ea5a6a4ff1222b95bad.jpg: 640x384 1 couplet, 852.0ms
image 8/23 /content/input_image/1b7b10992eb040d19fcfcadb745d1aa0.jpg: 640x256 1 couplet, 548.0ms
image 9/23 /content/input_image/1be6c13d483c41e6809a843fb9f1e913.jpg: 640x480 1 couplet, 1011.7ms
image 10/23 /content/input_image/1bf527e50d2447098ef82c510e5dab4d.jpg: 640x288 1 couplet, 923.9ms
image 11/23 /content/in

# Recognition

In [7]:
!pip install -q paddleocr --upgrade
!pip install -q paddlepaddle-gpu --upgrade

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 88.4/88.4 kB 1.3 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 407.5/407.5 kB 4.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 294.9/294.9 kB 15.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 908.3/908.3 kB 13.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 244.3/244.3 kB 12.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.4/3.4 MB 36.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 758.9/758.9 MB 1.2 MB/s eta 0:00:00


In [23]:
!python3 /content/recognition/ppocr/tools/infer/predict_rec.py --image_dir="./output_detection" --rec_model_dir='./weight/recognition_PPOCRLABEL'

/usr/local/lib/python3.10/dist-packages/paddle/base/framework.py:688: UserWarning: You are using GPU version Paddle, but your CUDA device is not set properly. CPU device will be used by default.
  warnings.warn(
[2024/08/27 16:09:24] ppocr WARNING: The first GPU is used for inference by default, GPU ID: 0
Traceback (most recent call last):
  File "/content/ppocr/tools/infer/predict_rec.py", line 881, in <module>
    main(utility.parse_args())
  File "/content/ppocr/tools/infer/predict_rec.py", line 838, in main
    text_recognizer = TextRecognizer(args)
  File "/content/ppocr/tools/infer/predict_rec.py", line 155, in __init__
    ) = utility.create_predictor(args, "rec", logger)
  File "/content/ppocr/tools/infer/utility.py", line 303, in create_predictor
    predictor = inference.create_predictor(config)
ValueError: (InvalidArgument) Device id must be less than GPU count, but received id is: 0. GPU count is: 0.
  [Hint: Expected id < GetGPUDeviceCount(), but received id:0 >= GetGPUDev